In [1]:
# import Pkg; Pkg.add("JLD2")
using DataFrames, CSV, Dates, JLD2

trips = CSV.read("../../data/trips/202210-bluebikes-tripdata.csv", DataFrame);
station_information = CSV.read("../../data/stations/station_information.csv", DataFrame);

# Trips 

In [2]:
# find unique station_id
station_ids = unique(station_information[:, :station_id]);
nb_stations = length(station_ids);
stations_match = Dict(zip(station_ids, 1:nb_stations));

# eliminate trips with station_id not in station_ids
trips = trips[in.(trips[:, :"start station id"], [Set(station_ids)]), :];
trips = trips[in.(trips[:, :"end station id"], [Set(station_ids)]), :];

# convert starttime and stoptime to DateTime
trips[!, :starttime] = DateTime.(trips[!, :starttime], "yyyy-mm-dd HH:MM:SS.ssss");
trips[!, :stoptime] = DateTime.(trips[!, :stoptime], "yyyy-mm-dd HH:MM:SS.ssss");

# extract day and hour from starttime and stoptime
trips[!, :startday] = Dates.day.(trips[!, :starttime]);
trips[!, :starthour] = Dates.hour.(trips[!, :starttime]);
trips[!, :stopday] = Dates.day.(trips[!, :stoptime]);
trips[!, :stophour] = Dates.hour.(trips[!, :stoptime]);

# create matrix of hourly trips between stations_match
nb_days = maximum(trips[:, :startday]);
trips_matrix = zeros(Int, nb_stations, nb_stations, nb_days*24);
for i in 1:size(trips, 1)
    trips_matrix[stations_match[trips[i, :"start station id"]], stations_match[trips[i, :"end station id"]], (trips[i, :startday]-1)*24+trips[i, :starthour]+1] += 1
end

In [12]:
# save stations_match dict to csv
CSV.write("../../data/stations/stations_match.csv", DataFrame(id=station_ids, row=1:nb_stations))

"../../data/stations/stations_match.csv"

In [3]:
jldsave("../../data/parameters/202210-trips-matrix.jld2", true; trips_matrix)

In [2]:
trips_matrix = load("../../data/parameters/202210-trips-matrix.jld2", "trips_matrix");

# Capacity

In [3]:
capacity = station_information[:, :capacity];

# Demand

In [4]:
function estimate_demand(trips_matrix)
    # if number of trips is more than 75% of capacity, corresponding number is quadrupled
    nb_stations, _, nb_hours = size(trips_matrix)
    demand = copy(trips_matrix)
    for k in 1:nb_hours

        # outgoing demand
        for i in 1:nb_stations
            if sum(trips_matrix[i, :, k]) > capacity[i]
                total_outgoing = sum(trips_matrix[i, :, k])
                for j in 1:nb_stations
                    demand[i, j, k] += ceil(trips_matrix[i, j, k] - trips_matrix[i, j, k] * capacity[i]/total_outgoing)
                end
            end
        end

        # incoming demand
        for j in 1:nb_stations
            if sum(trips_matrix[:, j, k]) > capacity[j]
                total_incoming = sum(trips_matrix[:, j, k])
                for i in 1:nb_stations
                    demand[i, j, k] += ceil(trips_matrix[i, j, k] - trips_matrix[i, j, k] * capacity[j]/total_incoming)
                end
            end
        end
    end
    return demand
end

estimate_demand (generic function with 1 method)

In [5]:
demand = estimate_demand(trips_matrix);

In [8]:
jldsave("../../data/parameters/202210-demand.jld2", true; demand)

In [ ]:
demand = load("../../data/parameters/202210-trips-matrix.jld2", "demand");

# Problem size reduction

In [ ]:
## Problem size reduction

# Order stations by demand

In [6]:
n_stations, _, nb_hours = size(demand)
# create empty list of size n_stations_test
stations_score = zeros(n_stations)
# for each station i, sum the number of bikes that gets in an out ot the station over the month
for i in 1:n_stations
    stations_score[i] += sum(demand[i,:,:]) + sum(demand[:,i,:])
end

In [7]:
# sort stations by score
stations_score_sorted = sortperm(stations_score, rev=true);

# keep only top 30 stations
top30_stations = stations_score_sorted[1:30];

# keep only top 50 stations
top50_stations = stations_score_sorted[1:50];

# keep only top 200 stations
top200_stations = stations_score_sorted[1:200];

# create dictionnary with key= station number in model, value: station_id in input file
stations_sorted_match = Dict()
for i in 1:n_stations
    stations_sorted_match[i] = stations_score_sorted[i]
end

# invert the dictionary stations_match
stations_match_inv = Dict(zip(values(stations_match), keys(stations_match)));

In [8]:
stations_sorted_match = hcat()
CSV.write("../../data/stations/stations_sorted_match.csv", stations_sorted_match |> Tables.table)

LoadError: MethodError: no method matching table(::Dict{Any, Any})
[0mClosest candidates are:
[0m  table([91m::AbstractVecOrMat[39m; header) at C:\Users\vasch\.julia\packages\Tables\Rophz\src\matrix.jl:63

In [16]:
# save stations ordering to CSV for Python maps
CSV.write("../../data/parameters/stations_sorted_match.csv", stations_score_sorted |> Tables.table)

"../../data/parameters/stations_sorted_match.csv"

In [10]:
# save stations ordering to JLD2 for Julia
jldsave("../../data/parameters/stations_sorted_match.jld2", true; stations_sorted_match)

In [55]:
# save top 200 stations to CSV 
CSV.write("../../data/stations/top200-stations.csv", DataFrame(station_id=top200_stations_id))

"../../data/stations/top200-stations.csv"

# Others

In [56]:
# sort stations by score inv
stations_score_sorted_incr = sortperm(stations_score, rev=false);
# keep only top 50 stations
worse_top100_stations = stations_score_sorted_incr[1:100];

In [57]:
# get the station_id of the worse top 100 stations
worse_top100_stations_id = [stations_match_inv[i] for i in worse_top100_stations];